# Imports

In [2]:
# for importing code
import sys, os

module_path = os.path.abspath(os.path.join("..", "src"))
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

Adding /Users/yonatanlou/dev/meaningful-transparency/src to sys.path


In [5]:
import pandas as pd
from constants import PROJECT_ROOT

train_df = pd.read_csv(PROJECT_ROOT / "datasets" / "train_test_datasets" / "train.csv")
samples = train_df[train_df["Biased"]==1].sample(10, random_state=42)
samples

,ID,Username,CreateDate,Biased,Keyword,Text,Year,n_words,n_words_no_urls_mentions_hashtags,extracted_urls,flag_link_only,flag_starts_with_mention,n_mentions_in_text,n_hashtags_in_text,n_urls_in_text,no_text_ratio,flag_empty_after_cleanup,flag_non_twitter_url,flag_twitter_url,Text_core
3056,1227888265887440897,Hasan08931491,2020-02-13 09:32:30+00:00,1,ZioNazi,"#Apartheid zionazist... Ethnic cleansing, land...",2020,23,19,['https://t.co/vYnnnQIG8U'],False,False,0,3,1,0.173913,False,False,True,"zionazist... Ethnic cleansing, land seizure, h..."
3417,1393637799597801474,mir_638,2021-05-15 18:41:58+00:00,1,Israel,RT @MohsinIslam: Israel is using international...,2021,37,34,['https://t.co/4lW3YWgGNz'],False,False,1,2,1,0.105263,False,False,True,RT : Israel is using internationally prohibite...
2167,1104601295523000323,mps051960,2019-03-10 04:34:06+00:00,1,Kikes,We'll soon hear Kikes in our time. She &amp; t...,2019,39,38,['https://t.co/BdnFSjeHRM'],False,False,0,0,1,0.025641,False,False,True,We'll soon hear Kikes in our time. She &amp; t...
3274,1395156785028222978,agarfilms,2021-05-19 23:17:52+00:00,1,Israel,RT @rafaelshimunov: The Israeli Interior Minis...,2021,42,41,['https://t.co/unpqNNGJ2z'],False,False,1,0,1,0.046512,False,False,True,RT : The Israeli Interior Ministry maintains a...
2316,1228143125643714561,Suket_T3k1,2020-02-14 02:25:13+00:00,1,ZioNazi,"#Apartheid zionazist...\nEthnic cleansing, lan...",2020,19,18,['https://t.co/JXXuu7oas7'],False,False,0,1,1,0.100000,False,False,True,"zionazist...\nEthnic cleansing, land seizure, ..."
1876,1395295293030207494,SallyfromSaar,2021-05-20 08:28:15+00:00,1,Jews,Such growing noble sentiments by US Jews/Israe...,2021,23,22,['https://t.co/RpWJHaKb91'],False,False,0,0,1,0.043478,False,False,True,Such growing noble sentiments by US Jews/Israe...
3001,1102700703682715648,iclayca,2019-03-04 22:41:49+00:00,1,Kikes,RT @USSLibincident: Ethno nationalism is only ...,2019,13,12,['https://t.co/bbwr7zFNlN'],False,False,1,0,1,0.142857,False,False,True,RT : Ethno nationalism is only considered ok w...
3058,1227929698040074240,GunGunG49169853,2020-02-13 12:17:08+00:00,1,ZioNazi,"#Apartheid zionazist... Ethnic cleansing, land...",2020,23,19,['https://t.co/vYnnnQIG8U'],False,False,0,3,1,0.173913,False,False,True,"zionazist... Ethnic cleansing, land seizure, h..."
3434,1392400276842323968,ZohaibKhalidPT,2021-05-12 08:44:30+00:00,1,Israel,Retweet and show the world if you agree with m...,2021,32,22,['https://t.co/hhQ7feQduJ'],False,False,0,10,1,0.333333,False,False,True,Retweet and show the world if you agree with m...
2238,1152595302274105345,kiwianna111,2019-07-20 15:05:08+00:00,1,ZioNazi,@reallygraceful More BS from the delusional pa...,2019,25,23,['https://t.co/3EiR0vHOlP'],False,True,1,0,1,0.080000,False,False,True,More BS from the delusional paranoid psychopat...


In [6]:
one_sample = samples.iloc[0]
one_sample

ID                                                                 1227888265887440897
Username                                                                 Hasan08931491
CreateDate                                                   2020-02-13 09:32:30+00:00
Biased                                                                               1
Keyword                                                                        ZioNazi
Text                                 #Apartheid zionazist... Ethnic cleansing, land...
Year                                                                              2020
n_words                                                                             23
n_words_no_urls_mentions_hashtags                                                   19
extracted_urls                                             ['https://t.co/vYnnnQIG8U']
flag_link_only                                                                   False
flag_starts_with_mention                   

In [11]:
from utils import llm, load_definitions
from constants import ANNOTATION_GLOB, CLASSIFIER_SYSTEM_ONE_DEF
all_annotations = load_definitions(str(ANNOTATION_GLOB))
one_def = "IHRA Definition"
system_prompt = CLASSIFIER_SYSTEM_ONE_DEF.replace(
        "<DEFINITION_NAME_PLACEHOLDER>", one_def
    )

In [13]:
print(system_prompt)

You are a careful text classifier.
You will be given:
- The definition of antisemitism by IHRA Definition
- a "text" string to classify
Task:
- You should classify if the text is antisemitic according to the given definition.
- The optional labels are: True (antisemitic by definition), False (not antisemitic by definition).
- Pick exactly one label.
- Output JSON only, no extra words.

Output schema:
{
  "answer": "<one key from annotations>",
  "description": "<1 to 3 sentences explaining the choice>"
}

Rules:
- Forget all other definitions of antisemitism, except the one by IHRA Definition.
- answer MUST be exactly True or False
- description must cite concrete cues from the text



In [16]:
user_prompt = (
        all_annotations[one_def]+
        "\n"
        + "\n\ntext:\n"
        + one_sample["Text"]
        + "\n\nRespond with JSON only matching the schema."
    )



print(user_prompt)

<annotation_guidelines>

<name>
IHRA (International Holocaust Remembrance Alliance) working definition of antisemitism
</name>

<definition>
Antisemitism is a certain perception of Jews, which may be expressed as hatred toward Jews. Rhetorical and physical manifestations of antisemitism are directed toward Jewish or non-Jewish individuals and/or their property, toward Jewish community institutions and religious facilities.
</defintion>

<guidelines>
To guide IHRA in its work, the following examples may serve as illustrations:

1. Manifestations might include the targeting of the state of Israel, conceived as a Jewish collectivity. However, criticism of Israel similar to that leveled against any other country cannot be regarded as antisemitic. Antisemitism frequently charges Jews with conspiring to harm humanity, and it is often used to blame Jews for “why things go wrong.” It is expressed in speech, writing, visual forms and action, and employs sinister stereotypes and negative charact

In [42]:
from constants import MAX_TOKENS
# MODEL = "openai/gpt-4o-mini" #Working
MODEL = "x-ai/grok-4" #Working


resp = llm(
                [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt[:300] +"\nText: " + one_sample["Text"]},
                ],
                MODEL,
                response_format={"type": "json_object"},
                temperature=0,
                max_tokens=MAX_TOKENS,
                logprobs= True,
                return_json=True,
                
            )

In [43]:
resp

{'id': 'gen-1757502435-SWoiEugq848b9KjS0OIC',
 'provider': 'xAI',
 'model': 'x-ai/grok-4',
 'object': 'chat.completion',
 'created': 1757502435,
 'choices': [{'logprobs': {'content': [{'token': ' First',
      'logprob': -0.055419736,
      'bytes': [32, 70, 105, 114, 115, 116],
      'top_logprobs': []},
     {'token': ',', 'logprob': 0, 'bytes': [44], 'top_logprobs': []},
     {'token': ' the',
      'logprob': -0.00016258826,
      'bytes': [32, 116, 104, 101],
      'top_logprobs': []},
     {'token': ' task',
      'logprob': -0.22431341,
      'bytes': [32, 116, 97, 115, 107],
      'top_logprobs': []},
     {'token': ' is',
      'logprob': -3.0994368e-06,
      'bytes': [32, 105, 115],
      'top_logprobs': []},
     {'token': ' to',
      'logprob': 0,
      'bytes': [32, 116, 111],
      'top_logprobs': []},
     {'token': ' classify',
      'logprob': -5.960463e-07,
      'bytes': [32, 99, 108, 97, 115, 115, 105, 102, 121],
      'top_logprobs': []},
     {'token': ' if',
  

In [44]:
resp.keys()

dict_keys(['id', 'provider', 'model', 'object', 'created', 'choices', 'system_fingerprint', 'usage'])

In [48]:
resp['choices'][0]["message"]

{'role': 'assistant', 'content': '{\n  "', 'refusal': None, 'reasoning': None}

In [38]:
import requests
url = "https://openrouter.ai/api/v1/models"
response = requests.get(url)
models = response.json()["data"]

In [ ]:
logprobs_models = [m for m in models if "logprobs" in m["supported_parameters"]]
not_pricy_models = [m for m in logprobs_models if m["pricing"]["prompt"] < ]
prompt
completion
pricing
import pprint
pprint.pprint(logprobs_models)

[{'architecture': {'input_modalities': ['text'],
                   'instruct_type': None,
                   'modality': 'text->text',
                   'output_modalities': ['text'],
                   'tokenizer': 'Other'},
  'canonical_slug': 'moonshotai/kimi-k2-0905',
  'context_length': 262144,
  'created': 1757021147,
  'description': 'Kimi K2 0905 is the September update of [Kimi K2 '
                 '0711](moonshotai/kimi-k2). It is a large-scale '
                 'Mixture-of-Experts (MoE) language model developed by '
                 'Moonshot AI, featuring 1 trillion total parameters with 32 '
                 'billion active per forward pass. It supports long-context '
                 'inference up to 256k tokens, extended from the previous '
                 '128k.\n'
                 '\n'
                 'This update improves agentic coding with higher accuracy and '
                 'better generalization across scaffolds, and enhances '
                 'frontend 

In [41]:
print(len(logprobs_models))

140
